In [96]:
import numpy as np
import plotly.graph_objects as go

#x,y,z = np.genfromtxt(r'dat.txt', unpack=True)
x = np.arange(100)
y = np.random.random(100)
z = np.random.random(100)

# Create figure
fig = go.Figure(
    data=[go.Scatter3d(x=[], y=[], z=[],
                     mode="markers",marker=dict(color="red", size=10))])
    
fig.update_layout(    
    scene = dict(
        
        xaxis=dict(range=[min(x), max(x)], autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False),
        )),


frames = [go.Frame(data= [go.Scatter3d(
                                       x=x[[k]], 
                                       y=y[[k]],
                                       z=z[[k]])],
                   
                   traces= [0],
                   name=f'frame{k}'      
                  )for k  in  range(len(x))]
fig.update(frames=frames),




fig.update_layout(updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))      ])])])


fig.show()


In [12]:
import pandas as pd
import io
import os
import glob
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np


@dataclass
class MocapLowerLandmark :
    LANK: np.ndarray = None # 0
    LASI: np.ndarray = None # 1
    LHEE: np.ndarray = None # 2
    LKNE: np.ndarray = None # 3
    LPSI: np.ndarray = None # 4
    LTHI: np.ndarray = None # 5
    LTIB: np.ndarray = None # 6
    LTOE: np.ndarray = None # 7
    RANK: np.ndarray = None # 8
    RASI: np.ndarray = None # 9
    RHEE: np.ndarray = None # 10
    RKNE: np.ndarray = None # 11
    RPSI: np.ndarray = None # 12
    RTHI: np.ndarray = None # 13
    RTIB: np.ndarray = None # 14
    RTOE: np.ndarray = None # 15
    
    CONN_PATH_LIST = [
         7,  2,  0,  3,  1,  4, 
        12,  9, 11,  8, 10, 15
    ]

    @staticmethod
    def calcKneeDegrees(mocap_df, idx = 0) :
        raw_val = mocap_df.loc[:, ~mocap_df.columns.isin(["Frame", "Time (Seconds)"]) ].to_numpy()
        raw_val = raw_val[idx].reshape(16, 3)

        rthai = raw_val[9] - raw_val[11]
        rcalf = raw_val[8] - raw_val[11]

        lthai = raw_val[1] - raw_val[3]
        lcalf = raw_val[0] - raw_val[3]

        rthai_unit = rthai / np.linalg.norm(rthai)
        rcalf_unit = rcalf / np.linalg.norm(rcalf)

        lthai_unit = lthai / np.linalg.norm(lthai)
        lcalf_unit = lcalf / np.linalg.norm(lcalf)

        rangle = np.arccos(np.dot(rthai_unit, rcalf_unit))
        langle = np.arccos(np.dot(lthai_unit, lcalf_unit))

        return langle, rangle

    @staticmethod
    def df2landmark(mocap_df, idx = 0) :
        raw_val = mocap_df.loc[:, ~mocap_df.columns.isin(["Frame", "Time (Seconds)"]) ].to_numpy()
        return raw_val[idx].reshape(16, 3)


def read_whole_data(trial_idx, experiment_idx) :
    FORCE_PLATE_1_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}_forceplate_1.csv"
    FORCE_PLATE_2_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}_forceplate_2.csv"
    MOCAP_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}.csv"
    EMG_PATH = f"/Volumes/HJP/CODES/mocap/data/d1/230626_Group7_trial{trial_idx}_{experiment_idx:03}_Trigno_2883.csv"

    
    with open(FORCE_PLATE_1_PATH, "r") as fp :
        lines = fp.readlines()
    raw_text = "".join(lines)
    table = raw_text.split("\n\n")[-1]
    fp1_df = pd.read_csv(io.StringIO(table))

    with open(FORCE_PLATE_2_PATH, "r") as fp :
        lines = fp.readlines()
    raw_text = "".join(lines)
    table = raw_text.split("\n\n")[-1]
    fp2_df = pd.read_csv(io.StringIO(table))

    with open(EMG_PATH, "r") as fp :
        lines = fp.readlines()
    raw_text = "".join(lines)
    table = raw_text.split("\n\n")[-1]
    emg_df = pd.read_csv(io.StringIO(table))

    with open(MOCAP_PATH, "r") as fp :
        lines = fp.readlines()
    mocap_df = pd.read_csv(io.StringIO( ''.join(lines[6:]) ))
    
    return mocap_df, emg_df, fp1_df, fp2_df

mocap_df, emg_df, fp1_df, fp2_df = read_whole_data(2, 1)

In [81]:
import numpy as np
import plotly.graph_objects as go

#x,y,z = np.genfromtxt(r'dat.txt', unpack=True)
x = np.arange(100)
y = np.random.random(100)
z = np.random.random(100)

# Create figure
fig = go.Figure(
    data = go.Scatter3d(
        x=[], y=[], z=[],
        mode="markers",
        marker=dict(color="red", size=10)
    )
)

# Frames
frames = [
    go.Frame(
        data = [
            go.Scatter3d(
                x=x[[k]],
                y=y[[k]],
                z=z[[k]]
            )
        ],
        traces= [0],
        name=f'frame{k}'      
    )for k  in  range(len(x)-1)
]
fig.update(frames=frames)


def frame_args(duration):
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }


sliders = [
    {
        "pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [
            {
                "args": [[f.name], frame_args(0)],
                "label": str(k),
                "method": "animate",
            } for k, f in enumerate(fig.frames)
        ]
    }
]

fig.update_layout(
    updatemenus = [
        {
            "buttons":[
                {
                    "args": [None, frame_args(50)],
                    "label": "Play", 
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "Pause", 
                    "method": "animate",
                }
            ],    
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }
    ],
    sliders=sliders
)

fig.update_layout(
    scene = dict(
        xaxis=dict(range=[min(x), max(x)],autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False)
    )
)

fig.update_layout(sliders=sliders)
fig.show()

In [70]:
MocapLowerLandmark.df2landmark(mocap_df, 100)

def plot3d_with_slider(mocap_df) :
    n_frames = len(mocap_df)
    points_list = np.array(list(map(
        lambda idx : MocapLowerLandmark.df2landmark(
            mocap_df, idx
        ),
        range(n_frames)
    )))
    
    points_list_ = np.nan_to_num(points_list.copy(), 0.1)

    xmin = points_list_[:, :, 0].min()
    xmax = points_list_[:, :, 0].max()
    ymin = points_list_[:, :, 1].min()
    ymax = points_list_[:, :, 1].max()
    zmin = points_list_[:, :, 2].min()
    zmax = points_list_[:, :, 2].max()

    
    connection_idx_list = MocapLowerLandmark.CONN_PATH_LIST
    line_color_list = ['red'] * (len(connection_idx_list))
    
    
    fig = go.Figure(
        data = go.Scatter3d(
            x=[], y=[], z=[],
            mode = 'lines+markers+text',
            marker=dict(color="red", size=2)
        )
    )
    frame_list = [
        go.Frame(
            data = list(map(
                lambda i, j, c : go.Scatter3d(
                    x = points[[i, j], 0],
                    y = points[[i, j], 1],
                    z = points[[i, j], 2],
                    mode = 'lines+markers+text',
                    line = dict(
                        color = c
                    ),
                    marker = dict(
                        color = c,
                        size = 2
                    )
                ),
                connection_idx_list[:-1],
                connection_idx_list[1:],
                line_color_list
            )),
            traces=[0],
            name=f"frame{fidx}"
        )
        for fidx, points in enumerate(points_list)
    ]
    fig.update(frames = frames)

    def frame_args(duration):
        return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

    sliders = [
        {
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [
                {
                    "args": [[f.name], frame_args(0)],
                    "label": str(k),
                    "method": "animate",
                } for k, f in enumerate(fig.frames)
            ]
        }
    ]

    fig.update_layout(
        updatemenus = [
            {
                "buttons":[
                    {
                        "args": [None, frame_args(50)],
                        "label": "Play", 
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "Pause", 
                        "method": "animate",
                    }
                ],    
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
        ],
        sliders=sliders
    )

    layout = go.Layout(
        scene = dict(
            camera = dict(
                eye = dict(x=-1, y = -1, z = 1)
            ),
            aspectmode="data"
        )
    )
    fig.layout = layout

    max_range = max(xmax-xmin, ymax-ymin, zmax, zmin)
    #max_range = np.max(np.ptp(points_list[0], axis=0))
    #xmin, ymin, zmin = points_list[0].min(axis=0)

    fig.update_layout(scene=dict(aspectmode='cube'))
    fig.update_layout(
        scene=dict(
            xaxis = dict(range=[xmin-0.1, xmin + max_range+ 0.1]),
            yaxis = dict(range=[ymin-0.1, ymin + max_range+ 0.1]),
            zaxis = dict(range=[zmin-0.1, zmin + max_range+ 0.1])
        )
    )



    fig.update_layout(sliders=sliders)
    fig.show()

plot3d_with_slider(mocap_df)

In [56]:
n_frames = len(mocap_df)
points_list = np.array(list(map(
    lambda idx : MocapLowerLandmark.df2landmark(
        mocap_df, idx
    ),
    range(n_frames)
)))

points_list_ = np.nan_to_num(points_list.copy(), 0.1)

xmin = points_list_[:, :, 0].min()
xmax = points_list_[:, :, 0].max()
ymin = points_list_[:, :, 1].min()
ymax = points_list_[:, :, 1].max()
zmin = points_list_[:, :, 2].min()
zmax = points_list_[:, :, 2].max()

xmin, xmax, ymin, ymax, zmin, zmax

(0.0, 0.575867, 0.0, 0.993897, 0.0, 0.836932)

In [58]:
max(1, 2, 3)

3

In [80]:
##

import numpy as np
import plotly.graph_objects as go

n_frames = len(mocap_df)
points_list = np.array(list(map(
    lambda idx : MocapLowerLandmark.df2landmark(
        mocap_df, idx
    ),
    range(n_frames)
)))
points_list_ = np.nan_to_num(points_list.copy(), 0.1)

xmin = points_list_[:, :, 0].min()
xmax = points_list_[:, :, 0].max()
ymin = points_list_[:, :, 1].min()
ymax = points_list_[:, :, 1].max()
zmin = points_list_[:, :, 2].min()
zmax = points_list_[:, :, 2].max()

connection_idx_list = MocapLowerLandmark.CONN_PATH_LIST
line_color_list = ['red'] * (len(connection_idx_list))

'''
lidx = 0
ridx = 100
jidx = 5
x = points_list[lidx:ridx, jidx, 0]
y = points_list[lidx:ridx, jidx, 1]
z = points_list[lidx:ridx, jidx, 2]

print(x)
print(y)
print(z)
'''

# Create figure
'''
fig = go.Figure(
    data = go.Scatter3d(
        x=[], y=[], z=[],
        mode="markers",
        marker=dict(color="red", size=10)
    )
)
'''

points = points_list[0]
fig = go.Figure(
    data = list(map(
        lambda i, j, c : go.Scatter3d(
            x = points[[i, j], 0],
            y = points[[i, j], 1],
            z = points[[i, j], 2],
            mode = 'lines+markers+text',
            line = dict(
                color = c
            ),
            marker = dict(
                color = c,
                size = 2
            )
        ),
        connection_idx_list[:-1],
        connection_idx_list[1:],
        line_color_list
    ))
)

# Frames
'''
frames = [
    go.Frame(
        data = [
            go.Scatter3d(
                x=x[[k]],
                y=y[[k]],
                z=z[[k]]
            )
        ],
        traces= [0],
        name=f'frame{k}'      
    )for k  in  range(len(x)-1)
]
fig.update(frames=frames)
'''
frames = [
    go.Frame(
        data = list(map(
            lambda i, j, c : go.Scatter3d(
                x = points[[i, j], 0],
                y = points[[i, j], 1],
                z = points[[i, j], 2],
            ),
            connection_idx_list[:-1],
            connection_idx_list[1:],
            line_color_list
        )),
        traces= [0],
        name=f'frame{k}'      
    )
    for k, points in enumerate(points_list)

]
#fig.update(frames=frames)



def frame_args(duration):
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }


sliders = [
    {
        "pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [
            {
                "args": [[f.name], frame_args(0)],
                "label": str(k),
                "method": "animate",
            } for k, f in enumerate(fig.frames)
        ]
    }
]

fig.update_layout(
    updatemenus = [
        {
            "buttons":[
                {
                    "args": [None, frame_args(50)],
                    "label": "Play", 
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "Pause", 
                    "method": "animate",
                }
            ],    
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }
    ],
    sliders=sliders
)

fig.update_layout(
    scene = dict(
        xaxis=dict(range=[min(x), max(x)],autorange=False),
        yaxis=dict(range=[min(y), max(y)], autorange=False),
        zaxis=dict(range=[min(z), max(z)], autorange=False)
    )
)


fig.update_layout(sliders=sliders)
fig.show()

In [73]:
for k, points in enumerate(points_list) :
    print(k, points.shape)

0 (16, 3)
1 (16, 3)
2 (16, 3)
3 (16, 3)
4 (16, 3)
5 (16, 3)
6 (16, 3)
7 (16, 3)
8 (16, 3)
9 (16, 3)
10 (16, 3)
11 (16, 3)
12 (16, 3)
13 (16, 3)
14 (16, 3)
15 (16, 3)
16 (16, 3)
17 (16, 3)
18 (16, 3)
19 (16, 3)
20 (16, 3)
21 (16, 3)
22 (16, 3)
23 (16, 3)
24 (16, 3)
25 (16, 3)
26 (16, 3)
27 (16, 3)
28 (16, 3)
29 (16, 3)
30 (16, 3)
31 (16, 3)
32 (16, 3)
33 (16, 3)
34 (16, 3)
35 (16, 3)
36 (16, 3)
37 (16, 3)
38 (16, 3)
39 (16, 3)
40 (16, 3)
41 (16, 3)
42 (16, 3)
43 (16, 3)
44 (16, 3)
45 (16, 3)
46 (16, 3)
47 (16, 3)
48 (16, 3)
49 (16, 3)
50 (16, 3)
51 (16, 3)
52 (16, 3)
53 (16, 3)
54 (16, 3)
55 (16, 3)
56 (16, 3)
57 (16, 3)
58 (16, 3)
59 (16, 3)
60 (16, 3)
61 (16, 3)
62 (16, 3)
63 (16, 3)
64 (16, 3)
65 (16, 3)
66 (16, 3)
67 (16, 3)
68 (16, 3)
69 (16, 3)
70 (16, 3)
71 (16, 3)
72 (16, 3)
73 (16, 3)
74 (16, 3)
75 (16, 3)
76 (16, 3)
77 (16, 3)
78 (16, 3)
79 (16, 3)
80 (16, 3)
81 (16, 3)
82 (16, 3)
83 (16, 3)
84 (16, 3)
85 (16, 3)
86 (16, 3)
87 (16, 3)
88 (16, 3)
89 (16, 3)
90 (16, 3)
91 (16, 3

In [84]:
np.linspace(0, 10, 100).reshape(-1, 2)

array([[ 0.        ,  0.1010101 ],
       [ 0.2020202 ,  0.3030303 ],
       [ 0.4040404 ,  0.50505051],
       [ 0.60606061,  0.70707071],
       [ 0.80808081,  0.90909091],
       [ 1.01010101,  1.11111111],
       [ 1.21212121,  1.31313131],
       [ 1.41414141,  1.51515152],
       [ 1.61616162,  1.71717172],
       [ 1.81818182,  1.91919192],
       [ 2.02020202,  2.12121212],
       [ 2.22222222,  2.32323232],
       [ 2.42424242,  2.52525253],
       [ 2.62626263,  2.72727273],
       [ 2.82828283,  2.92929293],
       [ 3.03030303,  3.13131313],
       [ 3.23232323,  3.33333333],
       [ 3.43434343,  3.53535354],
       [ 3.63636364,  3.73737374],
       [ 3.83838384,  3.93939394],
       [ 4.04040404,  4.14141414],
       [ 4.24242424,  4.34343434],
       [ 4.44444444,  4.54545455],
       [ 4.64646465,  4.74747475],
       [ 4.84848485,  4.94949495],
       [ 5.05050505,  5.15151515],
       [ 5.25252525,  5.35353535],
       [ 5.45454545,  5.55555556],
       [ 5.65656566,

In [101]:
# bottom up test

import numpy as np
import plotly.graph_objects as go

#x,y,z = np.genfromtxt(r'dat.txt', unpack=True)
x = np.linspace(0, 10, 100).reshape(-1, 2)
y = np.linspace(0, 10, 100).reshape(-1, 2)
z = np.linspace(0, 10, 100).reshape(-1, 2)

x = np.arange(100)
y = np.random.random(100)
z = np.random.random(100)

# Create figure
fig = go.Figure(
    data = go.Scatter3d(
        x=[], y=[], z=[],
        mode="markers",
        marker=dict(color="red", size=10)
    )
)

# Frames
frames = [
    go.Frame(
        data = [
            go.Scatter3d(
                x=x[k:k+2],
                y=y[k:k+2],
                z=z[k:k+2],
                mode="lines+markers"
            ),
            go.Scatter3d(
                x=x[k+1:k+3],
                y=y[k+1:k+3],
                z=z[k+1:k+3],
                mode="lines+markers"
            )
        
        ],
        traces= [0],
        name=f'frame{k}'      
    )for k  in  range(len(x)-3)
]
fig.update(frames=frames)


def frame_args(duration):
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }


sliders = [
    {
        "pad": {"b": 10, "t": 60},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [
            {
                "args": [[f.name], frame_args(0)],
                "label": str(k),
                "method": "animate",
            } for k, f in enumerate(fig.frames)
        ]
    }
]

fig.update_layout(
    updatemenus = [
        {
            "buttons":[
                {
                    "args": [None, frame_args(50)],
                    "label": "Play", 
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "Pause", 
                    "method": "animate",
                }
            ],    
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }
    ],
    sliders=sliders
)

fig.update_layout(
    scene = dict(
        xaxis=dict(range=[x.min(), x.max()], autorange=False),
        yaxis=dict(range=[y.min(), y.max()], autorange=False),
        zaxis=dict(range=[z.min(), z.max()], autorange=False)
    )
)

fig.update_layout(sliders=sliders)
fig.show()

In [92]:
x.min()

0.0